# 监控指标可视化

### 监控指标

> EVENTS

> IMAGES

> AUDIO

> HISTOGRAMS


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

SUMMARY_DIR = "/data/tmp/supervisor"
BATCH_SIZE = 100
TRAIN_STEMPS = 10000

# 生成变量监控信息并定义生成监控信息日志的操作
# var给出来需要记录的张量
# name给出来在可视化结果中显示的图表的名称，这个名称一般与变量名一致
def variable_summaries(var, name):
    # 将生成监控信息的操作放在同一个命名空间下
    with tf.name_scope('summaries'):
        # tf.histogram_summary记录张量中元素的取值分布
        # 不会立即执行，只有在sess.run明确调用这个操作时，才会真正生成并输出Summary protocol buffer
        tf.histogram_summary(name, var)
        
        # 计算变量的平均值，并定义生成平均值信息日志的操作
        # 'mean/' + name，其中mean为命名空间， ／ 时命名空间分隔符
        # name则给出当前监控指标属于哪个变量
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        
        # 计算变量的标准差
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.scalar_summary('stddev/' + name, stddev)

# 生成一层全链接神经网络
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    # 将同一层神经网络放在一个统一的命名空间下
    with tf.name_scope(layer_name):
        # 声明神经网络边上的权重，并调用生成权重监控信息日志
        with tf.name_scope('weights'):
            weights = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
            variable_summaries(weights, layer_name + '/weights')
        
        # 声明神经网络的偏置乡，并调用生成权重监控信息日志
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.constant(0.0, shape=[output_dim]))
            variable_summaries(biases, layer_name + 'biases')
        
        # 记录神经网络输出节点在经过激活函数之前的分布
        with tf.name_scope("Wx_plus_b"):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.histogram_summary(layer_name + '/pre_activations', preactivate)
        
        activations = act(preactivate, name='activation')
        
        # 记录神经网络输出节点在经过激活函数之后的分布
        # 对于layer1， 因为使用了ReLU作为激活函数，所以所有小于0的值都被设置为来0
        # 于是在激活函数后的layer1/activations图上的所有的值都是大于0的
        # 对于layer2，因为没有使用激活函数，所以layer2/activations和layer2/pre_activations一样
        tf.histogram_summary(layer_name + '/activations', activations)
        return activations
    
def main(_):
    mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist", one_hot=True)
    
    # 定义输入
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
    # 将输入向量还原成图片的像素矩阵
    # 通过tf.image_summary 将当前的图片信息写入日志
    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.image_summary('input', image_shaped_input, 10)
    
    hidden1 = nn_layer(x, 784, 500, 'layer1')
    y = nn_layer(hidden1, 500, 10, 'layer2', act=tf.identity)
    
    # 计算交叉熵，并生成交叉熵监控给日志
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
        tf.scalar_summary('cross entropy', cross_entropy)
        
    with tf.name_scope('train'):
        train_step = tf.train.AdadeltaOptimizer(0.001).minimize(cross_entropy)
        
    # 计算模型在当前给定数据上的正确率
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
        
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        tf.scalar_summary('accuracy', accuracy)
    
    # tf.merge_all_summaries整理所有的日志生成操作,不需要一一调用
    # 在tensorflow中只需要运行这个操作就可以将代码中的定义的所有的日志生成操作执行一次，从而将所有日志写入文件
    merged = tf.merge_all_summaries()
    
    with tf.Session() as sess:
        summary_writer = tf.train.SummaryWriter(SUMMARY_DIR, sess.graph)
        tf.initialize_all_variables().run()
        
        for i in range(TRAIN_STEMPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            # 运行虚拟烂步骤以及所有的日志生成操作
            summary, _ = sess.run([merged, train_step], feed_dict={x: xs, y_: ys})
            # 将所有日志写入文件
            summary_writer.add_summary(summary, i)
            
    summary_writer.close()
    
if __name__ == '__main__':
    tf.app.run()

Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz


SystemExit: 

/data/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
